## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Aklavik,CA,68.2191,-135.0107,48.20,57,20,2.30,few clouds
1,1,College,US,64.8569,-147.8028,72.00,68,40,0.00,scattered clouds
2,2,Dingle,PH,10.9995,122.6711,76.12,94,100,4.81,light rain
3,3,Faanui,PF,-16.4833,-151.7500,79.61,73,72,19.98,broken clouds
4,4,Hilo,US,19.7297,-155.0900,83.62,64,1,4.61,clear sky
...,...,...,...,...,...,...,...,...,...,...
673,673,Isetskoye,RU,56.4858,65.3226,51.17,91,98,8.68,overcast clouds
674,674,Tonantins,BR,-2.8731,-67.8022,83.71,71,72,2.06,moderate rain
675,675,Payakumbuh,ID,-0.2202,100.6308,67.46,77,95,2.53,overcast clouds
676,676,Nogliki,RU,51.8333,143.1667,62.91,99,100,4.05,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Dingle,PH,10.9995,122.6711,76.12,94,100,4.81,light rain
3,3,Faanui,PF,-16.4833,-151.7500,79.61,73,72,19.98,broken clouds
4,4,Hilo,US,19.7297,-155.0900,83.62,64,1,4.61,clear sky
10,10,Yinchuan,CN,38.4681,106.2731,79.86,50,70,4.43,broken clouds
15,15,Kavaratti,IN,10.5669,72.6420,82.49,79,100,22.41,light rain
...,...,...,...,...,...,...,...,...,...,...
665,665,Loiza,PR,18.4313,-65.8802,89.82,66,40,18.41,scattered clouds
667,667,Mukdahan,TH,16.5833,104.5000,75.29,86,71,2.93,broken clouds
668,668,Iracoubo,GF,5.4802,-53.2011,81.07,77,71,9.01,broken clouds
669,669,Cayenne,GF,4.9333,-52.3333,82.09,81,75,3.00,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                231
City                   231
Country                231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Current Description    231
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Dingle,PH,76.12,light rain,10.9995,122.6711,
3,Faanui,PF,79.61,broken clouds,-16.4833,-151.7500,
4,Hilo,US,83.62,clear sky,19.7297,-155.0900,
10,Yinchuan,CN,79.86,broken clouds,38.4681,106.2731,
15,Kavaratti,IN,82.49,light rain,10.5669,72.6420,
19,Komsomolskiy,UZ,77.72,broken clouds,40.4272,71.7189,
23,Adiake,CI,78.39,scattered clouds,5.2863,-3.3040,
25,Avarua,CK,75.25,light intensity shower rain,-21.2078,-159.7750,
26,Butaritari,KI,83.61,light rain,3.0707,172.7902,
28,Port Blair,IN,81.57,overcast clouds,11.6667,92.7500,


In [8]:
hotel_df.dtypes

City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Dingle,PH,76.12,light rain,10.9995,122.6711,SEGAYA PROPERTY
3,Faanui,PF,79.61,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
4,Hilo,US,83.62,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Yinchuan,CN,79.86,broken clouds,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch
15,Kavaratti,IN,82.49,light rain,10.5669,72.6420,Hotels in Lakshadweep Islands
...,...,...,...,...,...,...,...
665,Loiza,PR,89.82,scattered clouds,18.4313,-65.8802,Malibu Beachfront
667,Mukdahan,TH,75.29,broken clouds,16.5833,104.5000,Ban Kham Phung
668,Iracoubo,GF,81.07,broken clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
669,Cayenne,GF,82.09,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi


In [23]:
import numpy as np

In [24]:
hotel_df['Hotel Name'].replace("", np.nan, inplace=True)


In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna(axis=0,how="any",inplace=True)

In [30]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Dingle,PH,76.12,light rain,10.9995,122.6711,SEGAYA PROPERTY
3,Faanui,PF,79.61,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
4,Hilo,US,83.62,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Yinchuan,CN,79.86,broken clouds,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch
15,Kavaratti,IN,82.49,light rain,10.5669,72.6420,Hotels in Lakshadweep Islands
...,...,...,...,...,...,...,...
665,Loiza,PR,89.82,scattered clouds,18.4313,-65.8802,Malibu Beachfront
667,Mukdahan,TH,75.29,broken clouds,16.5833,104.5000,Ban Kham Phung
668,Iracoubo,GF,81.07,broken clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
669,Cayenne,GF,82.09,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi


In [27]:
hotel_df.loc[hotel_df["City"]=="Albany"]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))